In [1]:
from func_def import *

In [2]:
df = pd.read_csv('data_pre_processed.csv',index_col=[0]).reset_index(drop=True)
df.tail(3)

,url,event_url,date,result,fighter,opponent,title,method,round,time,format,f_head_sig_str,f_body_sig_str,f_leg_sig_str,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_sig_str_head_perc,f_sig_str_body_perc,f_sig_str_leg_perc,f_sig_str_dist_perc,f_sig_str_clinch_perc,f_sig_str_ground_perc,o_sig_str_head_perc,o_sig_str_body_perc,o_sig_str_leg_perc,o_sig_str_dist_perc,o_sig_str_clinch_perc,o_sig_str_ground_perc
7192,http://www.ufcstats.com/fight-details/ccee020b...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Johnny Rhode,David Levick,Open Weight Bout,KO/TKO,1,12:13,No Time Limit,9of15,1of1,1of1,1of1,1of1,9of15,4of5,0of0,0of0,1of2,2of2,1of1,0.81,0.09,0.09,0.09,0.09,0.81,1.0,0.0,0.0,0.25,0.5,0.25
7193,http://www.ufcstats.com/fight-details/4b9ae533...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Patrick Smit,Ray Wizar,Open Weight Bout,Submission,1,0:58,No Time Limit,0of0,1of1,0of0,0of0,1of1,0of0,0of0,0of0,1of1,1of1,0of0,0of0,0.00,1.00,0.00,0.00,1.00,0.00,0.0,0.0,1.0,1.00,0.0,0.00
7194,http://www.ufcstats.com/fight-details/4acab678...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Scott Morri,Sean Daughert,Open Weight Bout,Submission,1,0:20,No Time Limit,1of1,0of0,0of0,0of0,1of1,0of0,0of2,0of0,0of2,0of3,0of1,0of0,1.00,0.00,0.00,0.00,1.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00


In [3]:
# data enrichments
NEW_DATA_COLUMNS = ['win_perc','win_streak']
for column in NEW_DATA_COLUMNS:
    for prefix in ['f_','o_']:
        df[f'{prefix}{column}'] = [get_win_perc(fighter=df.loc[i,'fighter'],time=df.loc[i,'date'],data=df) for i in tqdm(range(len(df)))]
        df[f'{prefix}{column}'] = [get_win_streak(fighter=df.loc[i,'fighter'],time=df.loc[i,'date'],data=df) for i in tqdm(range(len(df)))]

df.head(3)

100%|██████████| 7195/7195 [00:22<00:00, 327.03it/s]


,url,event_url,date,result,fighter,opponent,title,method,round,time,format,f_head_sig_str,f_body_sig_str,f_leg_sig_str,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_sig_str_head_perc,f_sig_str_body_perc,f_sig_str_leg_perc,f_sig_str_dist_perc,f_sig_str_clinch_perc,f_sig_str_ground_perc,o_sig_str_head_perc,o_sig_str_body_perc,o_sig_str_leg_perc,o_sig_str_dist_perc,o_sig_str_clinch_perc,o_sig_str_ground_perc,f_win_perc,o_win_perc,f_win_streak,o_win_streak
0,http://www.ufcstats.com/fight-details/23a604f4...,http://www.ufcstats.com/event-details/c945adc2...,2023-09-23,L,Rafael Fiziev,Mateusz Gamrot,Lightweight Bout,KO/TKO,2,2:03,5 Rnd (5-5-5-5-5),4of14,12of16,2of2,18of32,0of0,0of0,11of31,2of5,3of4,13of35,1of1,2of4,0.22,0.66,0.11,1.00,0.00,0.00,0.68,0.12,0.18,0.81,0.06,0.12,1,1,1,1
1,http://www.ufcstats.com/fight-details/da1b37ed...,http://www.ufcstats.com/event-details/c945adc2...,2023-09-23,W,Bryce Mitchell,Dan Ige,Featherweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),13of22,7of14,3of4,17of29,1of3,5of8,13of47,3of6,2of2,16of51,2of3,0of1,0.56,0.30,0.13,0.73,0.04,0.21,0.72,0.16,0.11,0.88,0.11,0.00,0,0,0,0
2,http://www.ufcstats.com/fight-details/d8335b72...,http://www.ufcstats.com/event-details/c945adc2...,2023-09-23,W,Marina Rodrigue,Michelle Waterson-Gomez,Women's Strawweight Bout,KO/TKO,2,2:42,3 Rnd (5-5-5),47of70,22of30,1of2,12of23,53of73,5of6,7of23,5of7,1of2,4of17,7of9,2of6,0.67,0.31,0.01,0.17,0.75,0.07,0.53,0.38,0.07,0.30,0.53,0.15,0,0,0,0


In [4]:
# Pick features
#FEATURES = ['distance_sig_str','clinch_sig_str','ground_sig_str']
FEATURES = ['head_sig_str','body_sig_str','ground_sig_str']

feature_columns = [f'f_{item}' for item in FEATURES] + [f'o_{item}' for item in FEATURES]
print(f'Features Selected: {feature_columns}')

data = df[['date','fighter','opponent','result'] + feature_columns + [f'f_{feature}' for feature in NEW_DATA_COLUMNS] + [f'o_{feature}' for feature in NEW_DATA_COLUMNS]].copy()
for prefix in ['f_','o_']:
    for feature in FEATURES:
        data[f'{prefix}{feature}'] = pd.to_numeric([int(item.split('of')[0]) for item in df[f'{prefix}{feature}']])
        data[f'{prefix}{feature}_att'] = pd.to_numeric([int(item.split('of')[1]) for item in df[f'{prefix}{feature}']])

data.head(3)

Features Selected: ['f_head_sig_str', 'f_body_sig_str', 'f_ground_sig_str', 'o_head_sig_str', 'o_body_sig_str', 'o_ground_sig_str']


,date,fighter,opponent,result,f_head_sig_str,f_body_sig_str,f_ground_sig_str,o_head_sig_str,o_body_sig_str,o_ground_sig_str,f_win_perc,f_win_streak,o_win_perc,o_win_streak,f_head_sig_str_att,f_body_sig_str_att,f_ground_sig_str_att,o_head_sig_str_att,o_body_sig_str_att,o_ground_sig_str_att
0,2023-09-23,Rafael Fiziev,Mateusz Gamrot,L,4,12,0,11,2,2,1,1,1,1,14,16,0,31,5,4
1,2023-09-23,Bryce Mitchell,Dan Ige,W,13,7,5,13,3,0,0,0,0,0,22,14,8,47,6,1
2,2023-09-23,Marina Rodrigue,Michelle Waterson-Gomez,W,47,22,5,7,5,2,0,0,0,0,70,30,6,23,7,6


In [23]:
N = 3 #recent fights to include for certain stats
STATS = [item for item in FEATURES] + [f'{item}_att' for item in FEATURES]

print(f'Preparing {STATS} statistics:')
for stat in STATS:
    data[f'f_{stat}_mean'] = [get_mean_stat(fighter=data.loc[i,'fighter'],stat_col=f'f_{stat}',time=data.loc[i,'date'],data=data) for i in tqdm(range(len(data)))]
    data[f'o_{stat}_mean'] = [get_mean_stat(fighter=data.loc[i,'opponent'],stat_col=f'o_{stat}',time=data.loc[i,'date'],data=data) for i in tqdm(range(len(data)))]
    data[f'f_{stat}_last_{N}_mean'] = [get_last_n_stat_mean(fighter=data.loc[i,'fighter'],stat_col=f'f_{stat}',time=data.loc[i,'date'],data=data,n=N) for i in tqdm(range(len(data)))]
    data[f'o_{stat}_last_{N}_mean'] = [get_last_n_stat_mean(fighter=data.loc[i,'opponent'],stat_col=f'o_{stat}',time=data.loc[i,'date'],data=data,n=N) for i in tqdm(range(len(data)))]

data = data[['date','fighter','opponent','result'] + [column for column in data.columns if 'mean' in column]]
data.tail(3)

Preparing ['head_sig_str', 'body_sig_str', 'ground_sig_str', 'head_sig_str_att', 'body_sig_str_att', 'ground_sig_str_att'] statistics:



















































































































































































































































































100%|██████████| 7195/7195 [00:31<00:00, 230.93it/s]























































































































































































































































































































































































































100%|██████████| 7195/7195 [00:50<00:00, 141.30it/s]
























































































































































































































,date,fighter,opponent,result,f_head_sig_str_mean,o_head_sig_str_mean,f_head_sig_str_last_3_mean,o_head_sig_str_last_3_mean,f_body_sig_str_mean,o_body_sig_str_mean,f_body_sig_str_last_3_mean,o_body_sig_str_last_3_mean,f_ground_sig_str_mean,o_ground_sig_str_mean,f_ground_sig_str_last_3_mean,o_ground_sig_str_last_3_mean,f_head_sig_str_att_mean,o_head_sig_str_att_mean,f_head_sig_str_att_last_3_mean,o_head_sig_str_att_last_3_mean,f_body_sig_str_att_mean,o_body_sig_str_att_mean,f_body_sig_str_att_last_3_mean,o_body_sig_str_att_last_3_mean,f_ground_sig_str_att_mean,o_ground_sig_str_att_mean,f_ground_sig_str_att_last_3_mean,o_ground_sig_str_att_last_3_mean
7192,1994-03-11,Johnny Rhode,David Levick,W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7193,1994-03-11,Patrick Smit,Ray Wizar,W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7194,1994-03-11,Scott Morri,Sean Daughert,W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
N = 3 #recent fights to include for certain stats
STATS = [item for item in FEATURES] + [f'{item}_att' for item in FEATURES]

print(f'Preparing {STATS} mean & last {N} mean statistics:')
for stat in STATS:
    for index in tqdm(data.index):
        data.loc[index,f'f_{stat}_mean'] = get_mean_stat(fighter=data.loc[index,'fighter'],stat_col=f'f_{stat}',time=data.loc[index,'date'],data=data)
        data.loc[index,f'o_{stat}_mean'] = get_mean_stat(fighter=data.loc[index,'opponent'],stat_col=f'o_{stat}',time=data.loc[index,'date'],data=data)
        data.loc[index,f'f_{stat}_last_{N}_mean'] = get_last_n_stat_mean(fighter=data.loc[index,'fighter'],stat_col=f'f_{stat}',time=data.loc[index,'date'],data=data,n=N)
        data.loc[index,f'o_{stat}_last_{N}_mean'] = get_last_n_stat_mean(fighter=data.loc[index,'opponent'],stat_col=f'o_{stat}',time=data.loc[index,'date'],data=data,n=N)

data = data[['date','fighter','opponent','result'] + [column for column in data.columns if 'mean' in column]]
data.tail(3)

Preparing ['head_sig_str', 'body_sig_str', 'ground_sig_str', 'head_sig_str_att', 'body_sig_str_att', 'ground_sig_str_att'] mean & last 3 mean statistics:


 81%|████████  | 5803/7195 [01:22<00:19, 69.98it/s]


In [12]:
FEATURES
data.columns

Index(['date', 'fighter', 'opponent', 'result', 'f_head_sig_str',
       'f_body_sig_str', 'f_ground_sig_str', 'o_head_sig_str',
       'o_body_sig_str', 'o_ground_sig_str', 'f_win_perc', 'f_win_streak',
       'o_win_perc', 'o_win_streak', 'f_head_sig_str_att',
       'f_body_sig_str_att', 'f_ground_sig_str_att', 'o_head_sig_str_att',
       'o_body_sig_str_att', 'o_ground_sig_str_att', 'f_head_sig_str_mean',
       'o_head_sig_str_mean', 'f_head_sig_str_last_3_mean',
       'o_head_sig_str_last_3_mean', 'f_body_sig_str_mean',
       'o_body_sig_str_mean', 'f_body_sig_str_last_3_mean',
       'o_body_sig_str_last_3_mean', 'f_ground_sig_str_mean',
       'o_ground_sig_str_mean', 'f_ground_sig_str_last_3_mean',
       'o_ground_sig_str_last_3_mean', 'f_head_sig_str_att_mean',
       'o_head_sig_str_att_mean', 'f_head_sig_str_att_last_3_mean',
       'o_head_sig_str_att_last_3_mean', 'f_body_sig_str_att_mean',
       'o_body_sig_str_att_mean', 'f_body_sig_str_att_last_3_mean',
       'o_

In [24]:
for feature in FEATURES:
    #data[f'{feature}_dif'] = data[f'f_{feature}'] - data[f'o_{feature}']
    #data[f'{feature}_att_dif'] = data[f'f_{feature}_att'] - data[f'o_{feature}_att']
    #data[f'{feature}_mean_dif'] = data[f'f_{feature}_mean'] - data[f'o_{feature}_mean']
    #data[f'{feature}_att_mean_dif'] = data[f'f_{feature}_att_mean'] - data[f'o_{feature}_att_mean']
    data[f'{feature}_last_{N}_mean_dif'] = data[f'f_{feature}_last_{N}_mean'] - data[f'o_{feature}_last_{N}_mean']

for feature in NEW_DATA_COLUMNS:
    data[f'{feature}_dif'] = data[f'f_{feature}'] - data[f'o_{feature}']

data.head(3)

,date,fighter,opponent,result,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_win_perc,f_win_streak,o_win_perc,o_win_streak,f_distance_sig_str_att,f_clinch_sig_str_att,f_ground_sig_str_att,o_distance_sig_str_att,o_clinch_sig_str_att,o_ground_sig_str_att,f_distance_sig_str_last_3_mean,o_distance_sig_str_last_3_mean,f_clinch_sig_str_last_3_mean,o_clinch_sig_str_last_3_mean,f_ground_sig_str_last_3_mean,o_ground_sig_str_last_3_mean,f_distance_sig_str_att_last_3_mean,o_distance_sig_str_att_last_3_mean,f_clinch_sig_str_att_last_3_mean,o_clinch_sig_str_att_last_3_mean,f_ground_sig_str_att_last_3_mean,o_ground_sig_str_att_last_3_mean,distance_sig_str_last_3_mean_dif,clinch_sig_str_last_3_mean_dif,ground_sig_str_last_3_mean_dif,win_perc_dif,win_streak_dif
0,2023-09-23,Rafael Fiziev,Mateusz Gamrot,L,18,0,0,13,1,2,1,1,1,1,32,0,0,35,1,4,69.0,41.000000,6.333333,3.333333,0.333333,3.333333,142.333333,79.0,7.333333,4.000000,0.666667,4.000000,28.000000,3.000000,-3.0,0,0
1,2023-09-23,Bryce Mitchell,Dan Ige,W,17,1,5,16,2,0,0,0,0,0,29,3,8,51,3,1,24.0,48.000000,0.333333,4.333333,14.333333,2.333333,46.333333,112.0,0.333333,6.000000,19.333333,3.333333,-24.000000,-4.000000,12.0,0,0
2,2023-09-23,Marina Rodrigue,Michelle Waterson-Gomez,W,12,53,5,4,7,2,0,0,0,0,23,73,6,17,9,6,29.0,51.333333,5.333333,5.000000,7.000000,3.000000,90.000000,138.0,7.000000,7.333333,7.666667,4.333333,-22.333333,0.333333,4.0,0,0


In [25]:
#X = [f'{feature}_mean_dif' for feature in FEATURES] + [f'{feature}_att_mean_dif' for feature in FEATURES] + [f'{feature}_last_{N}_mean_dif' for feature in FEATURES ] + [f'{feature}_dif' for feature in NEW_DATA_COLUMNS]
X = [f'{feature}_last_{N}_mean_dif' for feature in FEATURES ] + [f'{feature}_dif' for feature in NEW_DATA_COLUMNS]
y = 'result'
X

['distance_sig_str_last_3_mean_dif',
 'clinch_sig_str_last_3_mean_dif',
 'ground_sig_str_last_3_mean_dif',
 'win_perc_dif',
 'win_streak_dif']

In [26]:
set_train_test(X_columns=X, y_columns=y, data=data, test_size=0.3, valid_size=0)
data.tail(3)

Data split: Data=(7195, 38), train_set=(5058, 38), validation_set=(0, 38), test_set=(2137, 38)


,set,date,fighter,opponent,result,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_win_perc,f_win_streak,o_win_perc,o_win_streak,f_distance_sig_str_att,f_clinch_sig_str_att,f_ground_sig_str_att,o_distance_sig_str_att,o_clinch_sig_str_att,o_ground_sig_str_att,f_distance_sig_str_last_3_mean,o_distance_sig_str_last_3_mean,f_clinch_sig_str_last_3_mean,o_clinch_sig_str_last_3_mean,f_ground_sig_str_last_3_mean,o_ground_sig_str_last_3_mean,f_distance_sig_str_att_last_3_mean,o_distance_sig_str_att_last_3_mean,f_clinch_sig_str_att_last_3_mean,o_clinch_sig_str_att_last_3_mean,f_ground_sig_str_att_last_3_mean,o_ground_sig_str_att_last_3_mean,distance_sig_str_last_3_mean_dif,clinch_sig_str_last_3_mean_dif,ground_sig_str_last_3_mean_dif,win_perc_dif,win_streak_dif
7192,test,1994-03-11,Johnny Rhode,David Levick,W,1,1,9,1,2,1,0,0,0,0,1,1,15,2,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
7193,test,1994-03-11,Patrick Smit,Ray Wizar,W,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
7194,train,1994-03-11,Scott Morri,Sean Daughert,W,0,1,0,0,0,0,0,0,0,0,0,1,0,3,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


### Logistic regression model train

In [27]:
# train model
X_train = data.loc[data['set'] == 'train', X].values
y_train = data.loc[data['set'] == 'train', y].values
    
# Create a logistic regression model
model = LogisticRegression(C=0.1,max_iter=1000)

# Fit the model to the training data
model.fit(X_train, y_train)
print(f"Logistic Regression(penalty={model.__dict__['penalty']},C={model.__dict__['C']},solver={model.__dict__['solver']},max_iter={model.__dict__['max_iter']})")

Logistic Regression(penalty=l2,C=0.1,solver=lbfgs,max_iter=1000)


In [28]:
# make predictions
data[f'{y}_pred'] = model.predict(data[X])
data[f'{y}_pred_proba'] = [max(item) for item in model.predict_proba(data[X])] #get max probability 
data.tail(3)

c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


,set,date,fighter,opponent,result,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_win_perc,f_win_streak,o_win_perc,o_win_streak,f_distance_sig_str_att,f_clinch_sig_str_att,f_ground_sig_str_att,o_distance_sig_str_att,o_clinch_sig_str_att,o_ground_sig_str_att,f_distance_sig_str_last_3_mean,o_distance_sig_str_last_3_mean,f_clinch_sig_str_last_3_mean,o_clinch_sig_str_last_3_mean,f_ground_sig_str_last_3_mean,o_ground_sig_str_last_3_mean,f_distance_sig_str_att_last_3_mean,o_distance_sig_str_att_last_3_mean,f_clinch_sig_str_att_last_3_mean,o_clinch_sig_str_att_last_3_mean,f_ground_sig_str_att_last_3_mean,o_ground_sig_str_att_last_3_mean,distance_sig_str_last_3_mean_dif,clinch_sig_str_last_3_mean_dif,ground_sig_str_last_3_mean_dif,win_perc_dif,win_streak_dif,result_pred,result_pred_proba
7192,test,1994-03-11,Johnny Rhode,David Levick,W,1,1,9,1,2,1,0,0,0,0,1,1,15,2,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,W,0.638893
7193,test,1994-03-11,Patrick Smit,Ray Wizar,W,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,W,0.638893
7194,train,1994-03-11,Scott Morri,Sean Daughert,W,0,1,0,0,0,0,0,0,0,0,0,1,0,3,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,W,0.638893


In [29]:
# Evaluate model
test_data = data[data.set=='test']
right_pred_test = len(test_data[test_data.result_pred == test_data.result])
all_test = len(test_data)
acc = right_pred_test/all_test
print(f'Accuracy = {acc}')

Accuracy = 0.6354702854468881


In [30]:
# Evaluate high probability predictions only
HIGH_PROBABILITY_SCORE = 0.8
test_data = data[(data.set=='test')&(data.result_pred_proba > HIGH_PROBABILITY_SCORE)]
right_pred_test = len(test_data[test_data.result_pred == test_data.result])
all_test = len(test_data)
acc = right_pred_test/all_test
print(f'Accuracy = {acc}')

Accuracy = 1.0


In [31]:
all_test

7

### Parameters Tuning

In [32]:
# train model
pt = data.copy()
X_train = pt.loc[pt['set'] == 'train', X].values
y_train = pt.loc[pt['set'] == 'train', y].values

# Create a logistic regression model
model = LogisticRegression(C=1,max_iter=1000,intercept_scaling=0.1)

# Fit the model to the training data
model.fit(X_train, y_train)

# make predictions
pt[f'{y}_pred'] = model.predict(pt[X])
pt[f'{y}_pred_proba'] = [max(item) for item in model.predict_proba(pt[X])] #get max probability 
print(f"Logistic Regression(penalty={model.__dict__['penalty']},C={model.__dict__['C']},solver={model.__dict__['solver']},max_iter={model.__dict__['max_iter']})")
pt.head(3)

Logistic Regression(penalty=l2,C=1,solver=lbfgs,max_iter=1000)


c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


,set,date,fighter,opponent,result,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_win_perc,f_win_streak,o_win_perc,o_win_streak,f_distance_sig_str_att,f_clinch_sig_str_att,f_ground_sig_str_att,o_distance_sig_str_att,o_clinch_sig_str_att,o_ground_sig_str_att,f_distance_sig_str_last_3_mean,o_distance_sig_str_last_3_mean,f_clinch_sig_str_last_3_mean,o_clinch_sig_str_last_3_mean,f_ground_sig_str_last_3_mean,o_ground_sig_str_last_3_mean,f_distance_sig_str_att_last_3_mean,o_distance_sig_str_att_last_3_mean,f_clinch_sig_str_att_last_3_mean,o_clinch_sig_str_att_last_3_mean,f_ground_sig_str_att_last_3_mean,o_ground_sig_str_att_last_3_mean,distance_sig_str_last_3_mean_dif,clinch_sig_str_last_3_mean_dif,ground_sig_str_last_3_mean_dif,win_perc_dif,win_streak_dif,result_pred,result_pred_proba
0,train,2023-09-23,Rafael Fiziev,Mateusz Gamrot,L,18,0,0,13,1,2,1,1,1,1,32,0,0,35,1,4,69.0,41.000000,6.333333,3.333333,0.333333,3.333333,142.333333,79.0,7.333333,4.000000,0.666667,4.000000,28.000000,3.000000,-3.0,0,0,W,0.632153
1,train,2023-09-23,Bryce Mitchell,Dan Ige,W,17,1,5,16,2,0,0,0,0,0,29,3,8,51,3,1,24.0,48.000000,0.333333,4.333333,14.333333,2.333333,46.333333,112.0,0.333333,6.000000,19.333333,3.333333,-24.000000,-4.000000,12.0,0,0,W,0.689023
2,train,2023-09-23,Marina Rodrigue,Michelle Waterson-Gomez,W,12,53,5,4,7,2,0,0,0,0,23,73,6,17,9,6,29.0,51.333333,5.333333,5.000000,7.000000,3.000000,90.000000,138.0,7.000000,7.333333,7.666667,4.333333,-22.333333,0.333333,4.0,0,0,W,0.651728


In [33]:
# Evaluate model
test_data = pt[pt.set=='test']
right_pred_test = len(test_data[test_data.result_pred == test_data.result])
all_test = len(test_data)
acc = right_pred_test/all_test
print(f'Accuracy = {acc}')

Accuracy = 0.6354702854468881
